In [ ]:
!pip install streamlit fastapi uvicorn wandb pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


## Creating the App

## Run the UI

In [25]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
from joblib import load
import os
import wandb
import sklearn
from joblib import load
from fastapi import FastAPI, HTTPException
from contextlib import asynccontextmanager
from pydantic import BaseModel
import wandb
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import joblib

app = FastAPI()
os.environ["WANDB_API_KEY"] = "6697a9d4d424fdce3f97021af11508c1941caf7e"

# Define input data schema


class PredictionRequest(BaseModel):
    song_name: str
    song_duration_ms: int
    acousticness: float
    danceability: float
    energy: float
    instrumentalness: float
    key: int
    liveness: float
    loudness: float
    audio_mode: int
    speechiness: float
    tempo: float
    time_signature: int
    audio_valence: float


# Load the model from WandB


def load_model_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB

        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        print(artifact)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        print(model_path)
        model = joblib.load(model_path)
        print(model)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")


def load_transformer_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB

        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        print(artifact)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        print(model_path)
        model = joblib.load(model_path)
        print(model)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")


def predict_popularity(
    acousticness,
    danceability,
    energy,
    instrumentalness,
    liveness,
    loudness,
    tempo,
    audio_valence,
):
    # Prepare the input data as a DataFrame

    inputs_dict = {
        "acousticness": float(acousticness),
        "danceability": float(danceability),
        "energy": float(energy),
        "instrumentalness": float(instrumentalness),
        "liveness": float(liveness),
        "loudness": float(loudness),
        "tempo": float(tempo),
        "audio_valence": int(audio_valence),
    }

    project_name = "MLOPS_SONG_POPULARITY_PREDICTION"  # Replace with your project name
    artifact_model_name = (
        "Model_Song_Popularity:latest"  # Replace with your artifact name
    )
    ml_model = load_model_from_wandb(project_name, artifact_model_name)

    artifact_transformer_name = (
        "Tranformer_Song_Popularity:latest"  # Replace with your artifact name
    )
    poly = load_transformer_from_wandb(project_name, artifact_transformer_name)

    df = pd.DataFrame(inputs_dict, index=[0])
    # Apply the same polynomial transformation

    transformed_features = poly.transform(df)  # Transform the input data the same

    # Call the model's prediction method

    prediction = ml_model.predict(transformed_features)

    # Return the prediction result

    return {prediction[0]}


# Function to define the app layout


def app_layout():
    st.title("Song Popularity Prediction")
    st.header(
        "Enter acousticness, danceability, energy, instrumentalness, liveness, loudness, tempo, audio_valence"
    )

    # Input fields for the features

    acousticness = st.number_input(
        "Acousticness:", min_value=1.02e-6, max_value=0.996, value=0.5
    )
    danceability = st.number_input(
        "Danceability:", min_value=0.0, max_value=0.987, value=0.49
    )
    energy = st.number_input("Energy:", min_value=0.00107, max_value=0.999, value=0.55)
    instrumentalness = st.number_input(
        "Instrumentalness:", min_value=0.0, max_value=0.997, value=0.1
    )
    liveness = st.number_input(
        "Liveness:", min_value=0.0109, max_value=0.986, value=0.5
    )
    loudness = st.number_input(
        "Loudness (dB):", min_value=-38.768, max_value=1.585, value=-15.0
    )
    tempo = st.number_input("Tempo:", min_value=0.0, max_value=242.318, value=120.0)
    audio_valence = st.number_input(
        "Audio Valence (0-1):", min_value=0.0, max_value=0.984, value=0.5
    )

    # Predict button

    if st.button("Predict Output"):
        prediction = predict_popularity(
            acousticness,
            danceability,
            energy,
            instrumentalness,
            liveness,
            loudness,
            tempo,
            audio_valence,
        )
        st.success(f"The predicted popularity value is: {prediction}")


if __name__ == "__main__":
    app_layout()


Overwriting app.py


In [15]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.168.251.39:8501

  Stopping...
  Stopping...
